In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
%matplotlib inline

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/My Drive/Projects/QuantumFlow/notebooks")
except:
    pass

if tf.test.gpu_device_name() == '/device:GPU:0':
    print('Found GPU')

import sys
sys.path.append('../')

from quantumflow.generate_potentials import generate_potentials
from quantumflow.calculus_utils import integrate, integrate_simpson, laplace
from quantumflow.numerov_solver import NumerovSolver

import ipywidgets as widgets
from IPython.display import display

In [ ]:
import pandas as pd

paper_coeff = pd.read_csv('../data/paper_potentials.txt', delimiter=' ')
paper_coeff.head()

In [ ]:
datadir = "../data"
dataset = "recreate_paper"

G = 500
length = 1
N = 4

a = paper_coeff[['a1', 'a2', 'a3']].values[:, np.newaxis, :]
c = paper_coeff[['b1', 'b2', 'b3']].values[:, np.newaxis, :] # b<->c listed wrong in the paper
b = paper_coeff[['c1', 'c2', 'c3']].values[:, np.newaxis, :]

np_x = np.linspace(0.0, length, G)
curves = -np.square(np_x[np.newaxis, :, np.newaxis] - b)/(2*np.square(c))
curves = -a*np.exp(curves)

np_potentials = np.sum(curves, -1)

h = (max(np_x) - min(np_x))/(G-1) # discretization width [bohr]

preview = 5

plt.figure(figsize=(20, 4))
plt.plot(np_x, np.transpose(np_potentials)[:, :preview]) # only plot first potentials
plt.show()

In [ ]:
progress = widgets.IntProgress(value=0, max=0, description='init...', bar_style='info', layout=widgets.Layout(width='92%'))
display(progress)

solver = NumerovSolver(G, h)

np_E, np_solutions = solver.solve_schroedinger(np_potentials, N, progress=progress)

if not os.path.exists(os.path.join(datadir, dataset)):
        os.makedirs(os.path.join(datadir, dataset))

with open(os.path.join(datadir, dataset, 'dataset_training.pkl'), 'wb') as f:
    pickle.dump([np_x, np_potentials, np_solutions, np_E], f)
    
print("dataset", dataset, "saved to", os.path.join(datadir, dataset))

In [ ]:
fig, axs = plt.subplots(1, preview, figsize=(20, 8))
for i, np_plot in enumerate(np_solutions[:preview]**2 + np_E[:preview, np.newaxis, :]):
    for n, np_plot_single in enumerate(np_plot.transpose()):
        axs[i].plot(np_x, np_potentials[i], 'k')
        axs[i].plot(np_x, np.ones(np_x.shape)*np_E[i, n], ':k')
        axs[i].plot(np_x, np_plot_single, 'C' + str(i%10))
        axs[i].set_ylim([np.min(np_potentials[:preview]), max(np.max(np_E[:preview]*1.1), 0.5)])
        if i == 0: axs[i].set_ylabel('energies, potential / hartree')
fig.suptitle('Numerov Solution Energies and Densities')
plt.show()

### recreate sample potential

In [ ]:
# parameters are found by gradient descent further down in the code
a = [4.02480191, 5.99455501, 10.76801402]
b = [0.52157438, 0.58645736, 0.57318963]
c = [0.07131018, 0.03986175, 0.05540802]

a = np.array(a)[np.newaxis, np.newaxis, :]
b = np.array(b)[np.newaxis, np.newaxis, :]
c = np.array(c)[np.newaxis, np.newaxis, :]

curves = -np.square(np_x[np.newaxis, :, np.newaxis] - b)/(2*np.square(c))
curves = -a*np.exp(curves)

np_sample = np.sum(curves, -1)

solver = NumerovSolver(G, h)

np_E_sample, np_solutions_sample = solver.solve_schroedinger(np_sample, N)

if not os.path.exists(os.path.join(datadir, dataset)):
        os.makedirs(os.path.join(datadir, dataset))

with open(os.path.join(datadir, dataset, 'dataset_sample.pkl'), 'wb') as f:
    pickle.dump([np_x, np_sample, np_solutions_sample, np_E_sample], f)
    
print("dataset dataset_sample.pkl saved to", os.path.join(datadir, dataset))

img = plt.imread("../data/sample_potential.png")
fig, ax = plt.subplots(figsize=(20, 12))
ax.imshow(img, extent=[0, 1, -40, 40])
plt.axis('auto')
plt.plot(np_x, np.transpose(np_sample))
plt.show()

In [ ]:
filtered_img = (img[:, :, 0] == 0)*1.0

kernel_size = 5
kernel = np.ones((kernel_size, kernel_size))/(kernel_size**2)
from scipy import ndimage
filtered_img = ndimage.convolve(filtered_img, kernel, mode='constant', cval=0.0)
filtered_img = (filtered_img > 0.92)*1.0
filtered_img[500:, 500:] = 0 # remove legend

y_mass = np.sum(filtered_img, axis=0)
y_data = np.sum(filtered_img*np.arange(filtered_img.shape[0])[:, np.newaxis], axis=0)/y_mass
y_data = -80*(y_data + 1/2 - filtered_img.shape[0]/2)/filtered_img.shape[0]
x_data = np.linspace(0, 1, filtered_img.shape[1])+0.5/filtered_img.shape[1]

filtered_img = 1 - filtered_img[:, :, np.newaxis]*np.ones((1, 1, 3))

fig, ax = plt.subplots(figsize=(20, 12))

ax.imshow(filtered_img, extent=[0, 1, -40, 40])
plt.axis('auto')
plt.plot(x_data, y_data)
plt.show()

In [ ]:
y_data -= np.nanmean(y_data[:100])
assert ~np.any(y_data > 0)

x_data = x_data[~np.isnan(y_data)]
y_data = y_data[~np.isnan(y_data)]

plt.figure(figsize=(20, 4))
plt.plot(x_data, y_data, '.k', markersize=1)
plt.show()

### solve the nonlinear optimization problem with tensorflow and gradient descent

In [ ]:
n_optimize = 10
seed = 0

a_minmax=(1.0, 10.0) # [hartree] 
b_minmax=(0.4, 0.6) # nomalized x
c_minmax=(0.03, 0.1) # normalized x

tf.reset_default_graph()
tf.random.set_random_seed(seed)

a = tf.Variable(tf.random.uniform((n_optimize, 3), minval=a_minmax[0], maxval=a_minmax[1], dtype=tf.float64), name="a")
b = tf.Variable(tf.random.uniform((n_optimize, 3), minval=b_minmax[0], maxval=b_minmax[1], dtype=tf.float64), name="b")
logc = tf.Variable(tf.random.uniform((n_optimize, 3), minval=np.log(c_minmax[0]), maxval=np.log(c_minmax[1]), dtype=tf.float64), name="c") # avoid negative values

fit_x = tf.constant(x_data)
fit_y = tf.constant(y_data)
fit_weights = tf.constant(x_data)*0 + 1

curves = -tf.square(tf.expand_dims(tf.expand_dims(fit_x, 1), 0) - tf.expand_dims(b, 1))/(2*tf.square(tf.expand_dims(tf.exp(logc), 1)))
curves = -tf.expand_dims(a, 1)*tf.exp(curves)

fit_potentials = tf.reduce_sum(curves, -1)

individual_loss = tf.reduce_mean(tf.square(fit_potentials - tf.expand_dims(fit_y, 0)), -1)
loss = tf.reduce_sum(individual_loss)

In [ ]:
optimizer = tf.train.AdamOptimizer(0.0002)
train = optimizer.minimize(loss)


np_a_list = []
np_b_list = []
np_c_list = []
loss_list = []

max_iterations = 100000

progress = widgets.IntProgress(value=0, max=max_iterations, description='optimizing', bar_style='info', layout=widgets.Layout(width='92%'))
display(progress)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(max_iterations):
        np_a, np_b, np_logc = sess.run([a, b, logc])
        np_c = np.exp(np_logc)
        
        np_a_list.append(np_a)
        np_b_list.append(np_b)
        np_c_list.append(np_c)
        
        np_loss, _ = sess.run([individual_loss, train])
        
        loss_list.append(np_loss)
    
        if step % 1000 == 0:
            progress.value = step
        
    np_a, np_b, np_logc = sess.run([a, b, logc])
    np_c = np.exp(np_logc)
    
    np_a_list.append(np_a)
    np_b_list.append(np_b)
    np_c_list.append(np_c)

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(np.log(loss_list))
plt.show()

In [ ]:
np_a = np.array(np_a_list)
np_b = np.array(np_b_list)
np_c = np.array(np_c_list)

np_x = np.linspace(0.0, length, G)
curves = -np.square(np_x[np.newaxis, :, np.newaxis] - np_b[-1, :, np.newaxis])/(2*np.square(np_c[-1, :, np.newaxis]))
curves = -np_a[-1, :, np.newaxis]*np.exp(curves)

np_potentials = np.sum(curves, -1)

print(np_a[0, :, :])
print(np_b[0, :, :])
print(np_c[0, :, :])
print('')
print(np_a[-1, :, :])
print(np_b[-1, :, :])
print(np_c[-1, :, :])

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(x_data, y_data, '.k', markersize=1)
plt.plot(np_x, np_potentials.transpose())
plt.show()

In [ ]:
index_min = np.argmin(loss_list[-1])
a_min = np_a_list[-1][index_min]
b_min = np_b_list[-1][index_min]
c_min = np_c_list[-1][index_min]
print(a_min)
print(b_min)
print(c_min)